Read in data:

In [ ]:
import pandas as pd
import string
import re



In [4]:
df = pd.read_csv('../webscraper/raw_data.csv')

df.head()  


,URL,HTML
0,https://deepmind.google/research/publications/,Publications - Google DeepMind ...
1,https://deepmind.google/research/publications/...,Publications - Google DeepMind ...
2,https://deepmind.google/research/publications/...,Automated Discovery of Interpretable Cognitive...
3,https://deepmind.google/research/publications/...,Scaling Pre-training to One Hundred Billion Da...
4,https://deepmind.google/research/publications/...,Publications - Google DeepMind ...


Clean the abstacts:

In [3]:

abstracts = df['HTML'].tolist()
abstracts = [' '.join(abstract.lower().translate(str.maketrans('', '', string.punctuation)).split()) for abstract in abstracts]

In [4]:

# regex to extract from HTML
def extract_abstract(text):
    match = re.search(r'(?i)abstract\s*(.*?)(?=\s*authors)', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None
def extract_publish_date(text):
    match = re.search(r'(?i)published\s+([0-9]{1,2}\s+\w+\s+[0-9]{4})', text)
    if match:
        return match.group(1).strip()
    return None
def extract_authors(text):
    match = re.search(r'Authors\s+(.*?)(?=\s*(External|Venue|Published|Footer|$))', text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()
    return None
def extract_title(text):
    match = re.search(r'([A-Z][^\n]{10,100}?)\s+Published', text)
    if match:
        return match.group(1).strip()
    return None


In [5]:
df['Title'] = df['HTML'].apply(extract_title)
df['Abstract'] = df['HTML'].apply(extract_abstract)
df['Publish_date'] = df['HTML'].apply(extract_publish_date)
df['Authors'] = df['HTML'].apply(extract_authors)


In [6]:
# checking extractions
print(df['URL'][3])
print(df['Title'][3])
print(df['Authors'][3])
print(df['Abstract'][3])
print(df['Publish_date'][3])

https://deepmind.google/research/publications/132991
Scaling Pre-training to One Hundred Billion Data for Vision Language Models
Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz, Zhe Li, Keran Rong, Xiaohua Zhai*  *
We provide an empirical investigation of the potential of pre-training vision-language models on an unprecedented scale: 100 billion examples. We find that model performance tends to saturate at this scale on many common Western-centric classification and retrieval benchmarks, such as COCO Captions. Nevertheless, tasks of cultural diversity achieve more substantial gains from the 100-billion scale web data, thanks to its coverage of long-tail concepts. Furthermore, we analyze the model's multilinguality and show gains in low-resource languages as well. In addition, we observe that reducing the size of the pretraining dataset via quality filters like using CLIP, typically used to enhance performance, may inadvertently reduce the cultural diversity represented even in large-scal

In [7]:
df.head(5)

,URL,HTML,Title,Abstract,Publish_date,Authors
0,https://deepmind.google/research/publications/,Publications - Google DeepMind ...,None,None,None,Marcus Hutter
1,https://deepmind.google/research/publications/...,Publications - Google DeepMind ...,None,None,None,"A. Feder Cooper, Christopher A. Choquette-Choo..."
2,https://deepmind.google/research/publications/...,Automated Discovery of Interpretable Cognitive...,Automated Discovery of Interpretable Cognitive...,A principal goal of computational neuroscience...,6 February 2025,"Pablo Castro Rivadeneira, Kim Stachenfeld, Kev..."
3,https://deepmind.google/research/publications/...,Scaling Pre-training to One Hundred Billion Da...,Scaling Pre-training to One Hundred Billion Da...,We provide an empirical investigation of the p...,11 February 2025,"Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz,..."
4,https://deepmind.google/research/publications/...,Publications - Google DeepMind ...,None,None,None,Marcus Hutter


In [9]:
df = df.dropna(subset=['Abstract'])

df.shape

(278, 6)

Remove dups from webscraping error:

In [22]:
df = pd.read_csv('./cleaned_url_data.csv')
print(df.shape)
df.head()

(278, 7)


,URL,HTML,Title,Abstract,Publish_date,Authors,KMeansTags
0,https://deepmind.google/research/publications/...,Automated Discovery of Interpretable Cognitive...,Automated Discovery of Interpretable Cognitive...,A principal goal of computational neuroscience...,6 February 2025,"Pablo Castro Rivadeneira, Kim Stachenfeld, Kev...","['programs', 'rl', 'discover', 'reward', 'mode..."
1,https://deepmind.google/research/publications/...,Scaling Pre-training to One Hundred Billion Da...,Scaling Pre-training to One Hundred Billion Da...,We provide an empirical investigation of the p...,11 February 2025,"Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz,...","['scale', 'vision', 'video', 'image', 'data']"
2,https://deepmind.google/research/publications/...,Delta Variances - Google DeepMind ...,Generate videos in Gemini and Whisk with Veo 2...,Decision makers may suffer from uncertainty in...,20 February 2025,"Simon Schmitt, John Shawe-Taylor, Hado van Has...","['neural', 'delta', 'weather', 'states', 'netw..."
3,https://deepmind.google/research/publications/...,Effective Kernel Fuzzing with Learned White-bo...,Veo 2 15 April 2025 Effectiv...,Kernel fuzzers rely heavily on program mutatio...,1 April 2025,"Sishuai Gong, Wang Rui, Deniz Altinbüken, Pedr...","['kernel', 'snowplow', 'coverage', 'programs',..."
4,https://deepmind.google/research/publications/...,TIPS: Text-Image Pretraining with Spatial awar...,Whisk with Veo 2 15 April 2025 ...,While image-text representation learning has b...,10 March 2025,"Kevis-Kokitsi Maninis, Kaifeng Chen, Soham Gho...","['image', 'tasks', 'dense', 'video', 'vision']"


In [33]:
df = df.drop_duplicates(subset=["Title"])
df.shape

(200, 7)

In [35]:

df = df.dropna(subset=['Title'])
df.shape

(199, 7)

There were around 80 papers with same Title and abstract but different URLS

In [36]:
df.to_csv('cleaned_url_data.csv', index=False)
